### Game mit Eventloop

In [1]:
import game
from ipywidgets import Output
from ipycanvas import Canvas
from IPython.display import display


layout = {'border': '1px solid black'}
out = Output(layout=layout)
canvas = Canvas(width=100, height=100, layout=layout)

state = {'pos': None}
key_move = {'ArrowUp': (0, -1),
            'ArrowDown': (0, 1),
            'ArrowLeft': (-1, 0),
            'ArrowRight': (1, 0),
            }


def grid2canvas(x, y, ncol, nrow):
    return x*canvas.width/ncol, y*canvas.height/nrow


@out.capture(clear_output=True)
def on_key_down(key, *flags):
    print(key)
    if key.startswith('Arrow'):
        dx, dy = key_move[key]
        game.move(dx, dy)
        move(game.state['head'], game.state['size'])
        if game.is_game_over():
            game_over()
    if key == 'n':
        game.new_game()
        new_game(game.state['head'], game.state['size'])


def new_game(head, size):
    x, y = grid2canvas(*head, *size)
    canvas.clear()
    canvas.fill_circle(x, y, 2)
    state['pos'] = x, y


def game_over():
    canvas.fill_text('Game over!', 10, 50)


def move(head, size):
    x, y = grid2canvas(*head, *size)
    x0, y0 = state['pos']
    canvas.stroke_lines([(x0, y0), (x, y)])
    state['pos'] = (x, y)


canvas.on_key_down(on_key_down)
display(canvas, out)
canvas.focus()

Canvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [2]:
import game
from ipywidgets import Output
from ipycanvas import Canvas
from IPython.display import display


layout = {'border': '1px solid black'}
out = Output(layout=layout)
canvas = Canvas(width=100, height=100, layout=layout)

state = {'pos': None}
key_move = {'ArrowUp': (0, -1),
            'ArrowDown': (0, 1),
            'ArrowLeft': (-1, 0),
            'ArrowRight': (1, 0),
            }


def grid2canvas(x, y, ncol, nrow):
    return x*canvas.width/ncol, y*canvas.height/nrow


@out.capture(clear_output=True)
def on_key_down(key, *flags):
    print(key)
    if key.startswith('Arrow'):
        dx, dy = key_move[key]
        game.move(dx, dy)
    if key == 'n':
        game.new_game()


def new_game(head, size):
    x, y = grid2canvas(*head, *size)
    canvas.clear()
    canvas.fill_circle(x, y, 2)
    state['pos'] = x, y


def game_over():
    canvas.fill_text('Game over!', 10, 50)


def move(head, size):
    x, y = grid2canvas(*head, *size)
    x0, y0 = state['pos']
    canvas.stroke_lines([(x0, y0), (x, y)])
    state['pos'] = (x, y)


def update(event, **kwargs):
    if event == 'new_game':
        new_game(**kwargs)
    if event == 'game_over':
        game_over()
    if event == 'move':
        move(**kwargs)


game.update = update
canvas.on_key_down(on_key_down)
display(canvas, out)
canvas.focus()

Canvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [5]:
import threading
import game
from ipywidgets import Output
from ipycanvas import Canvas
from IPython.display import display


layout = {'border': '1px solid black'}
out = Output(layout=layout)
canvas = Canvas(width=100, height=100, layout=layout)

state = {'pos': None}
key_move = {'ArrowUp': (0, -1),
            'ArrowDown': (0, 1),
            'ArrowLeft': (-1, 0),
            'ArrowRight': (1, 0),
            }

stop_event = threading.Event()
event_queue = []


def grid2canvas(x, y, ncol, nrow):
    return x*canvas.width/ncol, y*canvas.height/nrow


@out.capture(clear_output=True)
def on_key_down(key, *flags):
    print(key)
    if key.startswith('Arrow'):
        event_queue.insert(0, key)
    if key == 'n':
        game.new_game()


def new_game(head, size):
    x, y = grid2canvas(*head, *size)
    canvas.clear()
    canvas.fill_circle(x, y, 2)
    state['pos'] = x, y


def game_over():
    canvas.fill_text('Game over!', 10, 50)


def move(head, size):
    x, y = grid2canvas(*head, *size)
    x0, y0 = state['pos']
    canvas.stroke_lines([(x0, y0), (x, y)])
    state['pos'] = (x, y)


def update(event, **kwargs):
    # out.append_stdout(f'{event}, {kwargs}')
    if event == 'new_game':
        new_game(**kwargs)
        stop_event.clear()
        event_loop()
    if event == 'game_over':
        game_over()
        stop_event.set()
    if event == 'move':
        move(**kwargs)


def event_loop(last_event=None, count=0):
    if event_queue:
        key = event_queue.pop()
    else:
        key = last_event

    if key in key_move:
        dx, dy = key_move[key]
        game.move(dx, dy)

    if not stop_event.is_set():
        thread = threading.Timer(0.3, event_loop, args=(key, count+1))
        thread.name = f'Eventloop-{count}'
        thread.start()


game.update = update
canvas.on_key_down(on_key_down)

display(canvas, out)
canvas.focus()

Canvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [6]:
threading.active_count()

8